In [0]:
import json

from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import col, from_json

In [0]:
#connect to newly created Event Hub source1 (movies)
primaryKey_movies = "Endpoint=sb://db-eventhub-namespace.servicebus.windows.net/;SharedAccessKeyName=RootManageSharedAccessKey;SharedAccessKey=86ATi9XElcqYcspgAo8KNdXzMiWsSJbj++AEhA35NY8="
entityPath_movies = "EntityPath=eventhub-movies"

connectionString_movies = primaryKey_movies + ";" + entityPath_movies

ehConf_movies = {}

startingEventPosition_movies = {
  "offset": "-1",  
  "seqNo": -1,            
  "enqueuedTime": None,  
  "isInclusive": True
}

ehConf_movies['eventhubs.connectionString'] = sc._jvm.org.apache.spark.eventhubs.EventHubsUtils.encrypt(connectionString_movies)
ehConf_movies["eventhubs.startingPosition"] = json.dumps(startingEventPosition_movies)

In [0]:
#connect to newly created Event Hub source2 (ratings)
primaryKey_ratings = "Endpoint=sb://db-eventhub-namespace.servicebus.windows.net/;SharedAccessKeyName=RootManageSharedAccessKey;SharedAccessKey=86ATi9XElcqYcspgAo8KNdXzMiWsSJbj++AEhA35NY8="
entityPath_ratings = "EntityPath=eventhub-ratings"

connectionString_ratings = primaryKey_ratings + ";" + entityPath_ratings

ehConf_ratings = {}

startingEventPosition_ratings = {
  "offset": "-1",  
  "seqNo": -1,            
  "enqueuedTime": None,  
  "isInclusive": True
}

ehConf_ratings['eventhubs.connectionString'] = sc._jvm.org.apache.spark.eventhubs.EventHubsUtils.encrypt(connectionString_ratings)
ehConf_ratings["eventhubs.startingPosition"] = json.dumps(startingEventPosition_ratings)

In [0]:
# configure the schema for movies data
streaming_schema_movies = StructType([StructField("name", StringType(), True),
                                      StructField("year", IntegerType(), True),
                                      StructField("director", StringType(), True),
                                      StructField("writer", StringType(), True),
                                      StructField("star", StringType(), True)
                                     ])

In [0]:
# read stream movies data 
streaming_data_movies = spark.readStream \
                             .format("eventhubs") \
                             .options(**ehConf_movies) \
                             .load()

In [0]:
# get columns from json
streaming_data_movies = streaming_data_movies.select(from_json(col("body").cast("string"), streaming_schema_movies)) \
                                             .withColumnRenamed("from_json(CAST(body AS STRING))", "data") \
                                             .select(col('data.*')) 

streaming_data_movies.display()

name,year,director,writer,star
The Long Riders,1980,Walter Hill,Bill Bryden,David Carradine
Any Which Way You Can,1980,Buddy Van Horn,Stanford Sherman,Clint Eastwood
The Gods Must Be Crazy,1980,Jamie Uys,Jamie Uys,N!xau
Popeye,1980,Robert Altman,Jules Feiffer,Robin Williams
Ordinary People,1980,Robert Redford,Judith Guest,Donald Sutherland
Amar Akbar Anthony,null,null,null,null
The Long Riders,null,null,null,null
The Gods Must Be Crazy,null,null,null,null
Ordinary People,null,null,null,null


In [0]:
# configure schema for ratings 
streaming_schema_ratings = StructType([StructField("name", StringType(), True),
                                       StructField("rating", StringType(), True),
                                       StructField("score", FloatType(), True)
                                      ])

In [0]:
# read stream movies data
streaming_data_ratings = spark.readStream \
                             .format("eventhubs") \
                             .options(**ehConf_ratings) \
                             .load()

In [0]:
streaming_data_ratings = streaming_data_ratings.select(from_json(col("body").cast("string"), streaming_schema_ratings)) \
                                               .withColumnRenamed("from_json(CAST(body AS STRING))", "data") \
                                               .select(col('data.*'))   

streaming_data_ratings.display()

name,rating,score
Lagaan,PG,9.2
Andhadhun,PG,8.4
Airplane!,PG,7.7
Caddyshack,R,7.3
Friday the 13th,R,6.4
The Blues Brothers,R,7.9
Raging Bull,R,8.2
Star Wars: Episode V - The Empire Strikes Back,PG,8.2
Fame,R,6.6
Friday the 13th,R,6.1


In [0]:
# inner join of two streams
inner_join = streaming_data_movies.join(streaming_data_ratings, on=["name"])

inner_join.display()

name,year,director,writer,star,rating,score
Ordinary People,1980,Robert Redford,Judith Guest,Donald Sutherland,PG,8.1
Ordinary People,null,null,null,null,PG,8.1
The Long Riders,1980,Walter Hill,Bill Bryden,David Carradine,PG,7.1
The Long Riders,null,null,null,null,PG,7.1
The Gods Must Be Crazy,1980,Jamie Uys,Jamie Uys,N!xau,PG,8.6
The Gods Must Be Crazy,null,null,null,null,PG,8.6
Amar Akbar Anthony,null,null,null,null,PG,9.3


In [0]:
# additional operations combined with joins
join_and_filter = streaming_data_ratings.join(streaming_data_movies, on=["name"]) \
                                .where(streaming_data_ratings.rating == "PG")

join_and_filter.display()

name,rating,score,year,director,writer,star
Ordinary People,PG,8.1,null,null,null,null
Ordinary People,PG,8.1,1980,Robert Redford,Judith Guest,Donald Sutherland
The Long Riders,PG,7.1,null,null,null,null
The Long Riders,PG,7.1,1980,Walter Hill,Bill Bryden,David Carradine
The Gods Must Be Crazy,PG,8.6,null,null,null,null
The Gods Must Be Crazy,PG,8.6,1980,Jamie Uys,Jamie Uys,N!xau
Amar Akbar Anthony,PG,9.3,null,null,null,null


In [0]:
# try to perform left outer join on two streams 
not_supported_join = streaming_data_movies.join(streaming_data_ratings, on=["name"], how="leftOuter")

not_supported_join.display()

--------------------------------------------------------------------------- 
 AnalysisException Traceback (most recent call last)
 <command-1434252330829327> in <module> 
 2 not_supported_join = streaming_data_movies . join ( streaming_data_ratings , on = [ "name" ] , how = "leftOuter" ) 
 3 
 ----> 4 not_supported_join . display ( ) 

 /databricks/python_shell/dbruntime/monkey_patches.py in df_display (df, *args, **kwargs) 
 34 df . display ( ) is an alias for display ( df ) . Run help ( display ) for more information . 
 35 """
 ---> 36 display ( df , * args , ** kwargs ) 
 37 
 38 @ when_imported ( 'pyspark.sql' ) 

 /databricks/python_shell/scripts/PythonShellImpl.py in display (self, input, *args, **kwargs) 
 986 # exists and then if it does, whether it is actually streaming. 
 987 if hasattr ( input , 'isStreaming' ) and input . isStreaming : 
 --> 988 handleStreamingDataFrame ( input , self . sc , self . sqlContext , self . entry_point , kwargs ) 
 989 else : 
 990 if kwargs . get ( 'streamName' ) : 

 /databricks/python_shell/dbruntime/display.py in handleStreamingDataFrame (input, sc, sqlContext, entry_point, kwargs) 
 117 .DisplayHelper.getStreamName())
 118 
 --> 119 entry_point.getDriverSparkHooks().displayStreamingDataFrame(input._jdf, name, trigger,
 120 kwargs.get('checkpointLocation'))
 121 

 /databricks/spark/python/lib/py4j-0.10.9.1-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1302 
 1303 answer = self . gateway_client . send_command ( command ) 
 -> 1304 return_value = get_return_value(
 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 121 # Hide where the exception came from that shows a non-Pythonic 
 122 # JVM exception message. 
 --> 123 raise converted from None 
 124 else : 
 125 raise 

 AnalysisException : Stream-stream LeftOuter join between two streaming DataFrame/Datasets is not supported without a watermark in the join keys, or a watermark on the nullable side and an appropriate range condition;
Join LeftOuter, (name#69 = name#1344)
:- Project [data#67.name AS name#69, data#67.year AS year#70, data#67.director AS director#71, data#67.writer AS writer#72, data#67.star AS star#73]
: +- Project [from_json(CAST(body AS STRING))#65 AS data#67]
: +- Project [from_json(StructField(name,StringType,true), StructField(year,IntegerType,true), StructField(director,StringType,true), StructField(writer,StringType,true), StructField(star,StringType,true), cast(body#47 as string), Some(Etc/UTC)) AS from_json(CAST(body AS STRING))#65]
: +- StreamingRelation DataSource(org.apache.spark.sql.SparkSession@67881753,eventhubs,List(),None,List(),None,Map(eventhubs.connectionString -> +uQzV6R/bdPVogbaEgbQDO+B0aG+fd410pV+hHSUA7gFWco6+UvWjraCVX6ygZXvuQimx5knuCAiwcaN7A+z6arVqQVvtoRhTILa5RYBZOiIaCZkqwOr/8jLDp1r2QEP0+UC4FSAzw9RdnaPKq20IZ3StwwX3mONr6RJVQVfvzpNN+SGVNQ5GAgwjf448kk2BpHF6N2c5+ICSraC4NPEO73YRC0IFghXQWvbAH5eR7ud2i4Qk3fANKFqH7GWsr/LmzYjFp1M5H9dO3kvT5HEDw==, eventhubs.startingPosition -> {"offset": "-1", "seqNo": -1, "enqueuedTime": null, "isInclusive": true}),None), eventhubs, [body#47, partition#48, offset#49, sequenceNumber#50L, enqueuedTime#51, publisher#52, partitionKey#53, properties#54, systemProperties#55]
+- Project [data#1342.name AS name#1344, data#1342.rating AS rating#1345, data#1342.score AS score#1346]
 +- Project [from_json(CAST(body AS STRING))#1340 AS data#1342]
 +- Project [from_json(StructField(name,StringType,true), StructField(rating,StringType,true), StructField(score,FloatType,true), cast(body#1259 as string), Some(Etc/UTC)) AS from_json(CAST(body AS STRING))#1340]
 +- StreamingRelation DataSource(org.apache.spark.sql.SparkSession@67881753,eventhubs,List(),None,List(),None,Map(eventhubs.connectionString -> +uQzV6R/bdPVogbaEgbQDO+B0aG+fd410pV+hHSUA7gFWco6+UvWjraCVX6ygZXvuQimx5knuCAiwcaN7A+z6arVqQVvtoRhTILa5RYBZOiIaCZkqwOr/8jLDp1r2QEP0+UC4FSAzw9RdnaPKq20IZ3StwwX3mONr6RJVQVfvzpNN+SGVNQ5GAgwjf448kk2

NB: neither outer join is working on stream side, unless watermark is specified